<h1>Using <i style="color:red;">SQLAlchemy</i>'s ORM in <i style="color:green;">Django</i></h1>

<h3><i>by Jack Camier, Python Developer</i></h3>

# Django is one of the most popular web frameworks in Python. As of this talk, 2027 companies reportedly use Django in their tech stacks, including YouTube, Bitbucket, Dropbox, Instagram, Pinterest, Udemy just to name a few...
https://stackshare.io/django

In [ ]:
%%HTML
<img src="django_logo.png" width="300"/>

https://www.djangoproject.com/

## Personally, I am a big fan and use it often. However, one aspect of Django that I don't like that much is its ORM. I am not alone and have heard from developers their frustrations with the ORM. It works great for simple projects and queries but whenever you try to do something more complex, you can run into problems.

## In this quick tutorial, I will show you a way you can use SQLAlchemy's ORM in conjunction with your Django Projects.

## We will use the Django tutorial as our baseline to see how this can work.
https://docs.djangoproject.com/en/3.0/intro/tutorial01/

## I am using the latest Django version as of this writing 3.0.5

In [ ]:
import django

django.VERSION

## You will need to use the django-extensions to run this in the Jupyter Notebook. Not necessary for your local project

In [ ]:
pip install django-extensions

## Steps from the online tutorial

In [ ]:
!django-admin startproject mysite